In [ ]:
##'final score' לכל מתנדב עבור כל קטגוריה.  הסבר על הקוד- בקוד הזה מתקבל לנו הציון הסופי שהמתנדב מקבל מהמשוואה שמכילה את ממוצע זמן תיקון תקלה, דירוג לקוחות ואחוזי הצלחה. בטבלה הסופית שאנחנו מקבלים נראה את הציון הסופי בעמודה

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
user_data_df = pd.read_csv('/content/drive/MyDrive/final project/קודים גמורים /Data/user_data_table.csv.csv')

In [ ]:
### לא למחוק את השורה הזאת
user_data_df = user_data_df.dropna(subset=['ממוצע דירוג לקוח '])

In [ ]:
# שלב 2: חישוב הממוצעים של כל עמודה לפי קטגוריות עבור העמודות הרלוונטיות בלבד
columns_to_average = ['ממוצע זמן טיפול בתקלה', 'ממוצע דירוג לקוח ', 'אחוזי הצלחה']
averages = user_data_df.groupby('קטגוריה')[columns_to_average].transform('mean')

# שלב 3: חישוב היחסים בין כל ערך לממוצע של העמודה המתאימה בקטגוריה
# הערה: בשביל ממוצע זמן טיפול בתקלה, נשתמש ביחס הפוך
user_data_df['t_ratio'] = averages['ממוצע זמן טיפול בתקלה'] / user_data_df['ממוצע זמן טיפול בתקלה']
user_data_df['r_ratio'] = user_data_df['ממוצע דירוג לקוח '] / averages['ממוצע דירוג לקוח ']
user_data_df['w_ratio'] = user_data_df['אחוזי הצלחה'] / averages['אחוזי הצלחה']

# שלב 4: מתן ציונים לפי הסולם שניתן
def score(value):
    if 0 <= value <= 0.4:
        return 1
    elif 0.4 < value <= 0.7:
        return 2
    elif 0.7 < value <= 1:
        return 3
    elif 1 < value <= 1.3:
        return 4
    else:
        return 5

# שימוש ב- vectorization לשיפור היעילות
score_vectorized = np.vectorize(score)

user_data_df['t_score'] = score_vectorized(user_data_df['t_ratio'])
user_data_df['r_score'] = score_vectorized(user_data_df['r_ratio'])
user_data_df['w_score'] = score_vectorized(user_data_df['w_ratio'])

# שלב 5: חישוב הציון הכולל של כל מתנדב לפי המשוואה
# נניח שהעמודה שמכילה את מספר האירועים נקראת 'num_events'
user_data_df['rank'] = (user_data_df['t_score'] * 0.5 + user_data_df['r_score'] * 0.1 + user_data_df['w_score'] * 0.4) * np.log1p(user_data_df['כמות אירועים'])

# שלב 6: נרמול הציונים בין 1 ל-100 עבור כל קטגוריה בנפרד
def normalize_category(group):
    min_rank = group['rank'].min()
    max_rank = group['rank'].max()
    group['final score'] = 1 + ((group['rank'] - min_rank) * (100 - 1) / (max_rank - min_rank))
    return group

user_data_df = user_data_df.groupby('קטגוריה').apply(normalize_category)
user_data_df


/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


מזהה כונן      קטגוריה  ממוצע זמן טיפול בתקלה  \
קטגוריה                                                              
אחר         10      0001 שלמה           אחר              24.000000   
            20     0003 שלומי           אחר              14.000000   
            25     0004 אליהו           אחר               1.000000   
            48       0009 דדי           אחר              25.280388   
            49      0009 יוסי           אחר              12.640194   
...                        ...          ...                    ...   
שמן-מים-דלק 21757  9980 ליאור   שמן-מים-דלק              21.434601   
            21765   9983 אורי   שמן-מים-דלק              13.434601   
            21772    9987 גיל   שמן-מים-דלק              23.869202   
            21776    9991 אסף   שמן-מים-דלק              24.000000   
            21785  9992 פינחס   שמן-מים-דלק              18.267934   

                   כמות אירועים  אירועים מוצלחים  אחוזי הצלחה  \
קטגוריה                                                         
אחר         10                1              1.0        100.0   
            20                1              1.0        100.0   
            25                1              1.0        100.0   
            48                1              1.0        100.0   
            49                6              5.0        100.0   
...                         ...              ...          ...   
שמן-מים-דלק 21757             2              1.0        100.0   
            21765             4              3.0         75.0   
            21772             1              1.0        100.0   
            21776             1              1.0        100.0   
            21785             6              6.0        100.0   

                   ממוצע דירוג לקוח     t_ratio   r_ratio   w_ratio  t_score  \
קטגוריה                                                                        
אחר         10              2.944444   0.975470  0.980425  1.159445        3   
            20              3.250000   1.672234  1.082168  1.159445        5   
            25              3.454545  23.411283  1.150276  1.159445        5   
            48              1.000000   0.926065  0.332975  1.159445        3   
            49              2.869565   1.852130  0.955493  1.159445        5   
...                              ...        ...       ...       ...      ...   
שמן-מים-דלק 21757           3.000000   1.075210  0.995664  1.103147        4   
            21765           3.024390   1.715474  1.003759  0.827360        5   
            21772           2.854167   0.965542  0.947264  1.103147        3   
            21776           3.270270   0.960279  1.085364  1.103147        3   
            21785           3.090308   1.261593  1.025636  1.103147        4   

                   r_score  w_score      rank  final score  
קטגוריה                                                     
אחר         10           3        4  2.356700    10.182565  
            20           4        4  3.119162    14.574226  
            25           4        4  3.119162    14.574226  
            48           1        4  2.218071     9.384081  
            49           3        4  8.562005    45.924147  
...                    ...      ...       ...          ...  
שמן-מים-דלק 21757        3        4  4.284588    15.516926  
            21765        4        3  6.598695    25.054837  
            21772        3        4  2.356700     7.570873  
            21776        4        4  2.426015     7.856563  
            21785        4        4  7.783641    29.938751  

[21804 rows x 15 columns]

In [ ]:
user_data_df[user_data_df['קטגוריה'] == 'שמן-מים-דלק'].sort_values(by='final score')

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


מזהה כונן      קטגוריה  ממוצע זמן טיפול בתקלה  \
קטגוריה                                                              
שמן-מים-דלק 13257  5842 איילת   שמן-מים-דלק              94.000000   
            6948     2860 משה   שמן-מים-דלק              65.000000   
            6615   2702 שמעון   שמן-מים-דלק             104.000000   
            4122    1580 אורן   שמן-מים-דלק              65.000000   
            6013    2420 שלמה   שמן-מים-דלק              76.000000   
...                        ...          ...                    ...   
            780     0231 יוסף   שמן-מים-דלק              12.582157   
            17074   7628 שרגא   שמן-מים-דלק              14.475824   
            7203    3005 יוסי   שמן-מים-דלק              17.850053   
            6470   2632 בצלאל   שמן-מים-דלק              13.414527   
            1099    0340 רינו   שמן-מים-דלק              22.280371   

                   כמות אירועים  אירועים מוצלחים  אחוזי הצלחה  \
קטגוריה                                                         
שמן-מים-דלק 13257             1              0.0     0.000000   
            6948              1              0.0     0.000000   
            6615              1              0.0     0.000000   
            4122              1              0.0     0.000000   
            6013              1              0.0     0.000000   
...                         ...              ...          ...   
            780              69             69.0   100.000000   
            17074           113            111.0    98.230088   
            7203            437            360.0    82.379863   
            6470            136            132.0    97.058824   
            1099            574            572.0    99.651568   

                   ממוצע דירוג לקוח    t_ratio   r_ratio   w_ratio  t_score  \
קטגוריה                                                                       
שמן-מים-דלק 13257           2.000000  0.245178  0.663776  0.000000        1   
            6948            2.200000  0.354565  0.730154  0.000000        1   
            6615            2.946809  0.221603  0.978011  0.000000        1   
            4122            2.600000  0.354565  0.862909  0.000000        1   
            6013            3.250000  0.303246  1.078636  0.000000        1   
...                              ...       ...       ...       ...      ...   
            780             2.990521  1.831698  0.992518  1.103147        5   
            17074           2.932536  1.592083  0.973274  1.083622        5   
            7203            2.954525  1.291128  0.980572  0.908771        4   
            6470            2.959831  1.718041  0.982333  1.070701        5   
            1099            3.001735  1.034395  0.996240  1.099303        4   

                   r_score  w_score       rank  final score  
קטגוריה                                                      
שמן-מים-דלק 13257        2        1   0.762462     1.000000  
            6948         3        1   0.831777     1.285690  
            6615         3        1   0.831777     1.285690  
            4122         3        1   0.831777     1.285690  
            6013         4        1   0.901091     1.571380  
...                    ...      ...        ...          ...  
            780          3        4  18.693379    74.904735  
            17074        3        4  20.839273    83.749332  
            7203         3        3  21.287766    85.597858  
            6470         3        4  21.647916    87.082265  
            1099         3        4  24.782043   100.000000  

[2110 rows x 15 columns]

In [ ]:
output_file = 'weight.csv'
user_data_df.to_csv(output_file, index=False)